In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
from PIL import Image
%matplotlib inline
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
#Cell to verify data formats
test3 = pd.read_csv('/Users/JosephVele/Downloads/all/train_photo_to_biz_ids.csv')
test4 = test3[test3['photo_id']==5].iloc[:,3:12].values[0].tolist()
test3.iloc[0,3:12].values

array([0, 1, 1, 0, 1, 1, 1, 1, 0], dtype=object)

In [2]:
#Testing cell not used - Used for testing when creating data loader
path = '/Users/JosephVele/Downloads/all/train_photo_to_biz_ids.csv'
train_data = pd.read_csv(path,usecols=range(1),dtype = 'str') #nameframe
label_data=pd.read_csv(path,sep=",", usecols=[2], nrows=1000) #label_frame
root = '/Users/JosephVele/Downloads/all/train_photos'
transform = transforms.Compose([transforms.ToTensor()])

pic_name = os.path.join(root, train_data.iloc[1, 0]+'.jpg')
pic = Image.open(pic_name)
transform(pic)

np.array(label_data.iloc[1,0].split()).astype('int')


array([3, 6])

In [65]:
class yelpDataset(torch.utils.data.Dataset):
    
    def __init__(self,text_file,root_dir, transform):
        """
        Args:
            text_file(string): path to text file
            root_dir(string): directory with all train images
        """
        
        self.name_frame = pd.read_csv(text_file,sep=",",usecols=range(1),dtype = 'str',nrows = 10000)
        self.label_frame = pd.read_csv(text_file,sep=",",  nrows=10000)
        self.root_dir = root_dir
        self.transform = transform
                                       
    def __len__(self):
        return len(self.name_frame)

    def __getitem__(self, idx):
        #photoid = self.name_frame.iloc[idx, 0]
        img_name = os.path.join(self.root_dir, self.name_frame.iloc[idx, 0]  +'.jpg')
        #print(img_name)
        image = Image.open(img_name)
        image = image.convert('RGB')
        image = self.transform(image) 
        labels = self.label_frame.iloc[idx,3:12].values
        labels = np.array(labels)
        labels= torch.from_numpy(labels.astype('int'))
        #print(labels)
        #labels = self.label_frame.iloc[idx,0]
        #labels = labels.reshape(-1, 2)
        sample = {'image': image, 'labels': labels}
        
        return sample
    

In [64]:
yelpTrainSet = yelpDataset(text_file ='/Users/JosephVele/Downloads/all/train_photo_to_biz_ids.csv',
                           root_dir = '/Users/JosephVele/Downloads/all/train_photos',
                          transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean = [0.485, 0.456, 0.406],
                                                              std = [0.229, 0.224, 0.225])]))

yelpTrainLoader = torch.utils.data.DataLoader(yelpTrainSet,batch_size=10,shuffle=True, num_workers=0)

yelpTestSet = yelpDataset(text_file ='/Users/JosephVele/Downloads/all/test_photo_to_biz_ids.csv',
                          root_dir = '/Users/JosephVele/Downloads/all/train_photos',
                          transform = transforms.Compose([transforms.Resize((224,224)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean = [0.485, 0.456, 0.406],
                                                              std = [0.229, 0.224, 0.225])]))

yelpTestLoader = torch.utils.data.DataLoader(yelpTrainSet,batch_size=10,shuffle=True, num_workers=0)



In [26]:
for i_batch,sample_batched in enumerate(yelpTrainLoader,0):
    batch_size =sample_batched['image'].shape[0]
    image_size = sample_batched['image'].shape[1:4]
    n_labels = sample_batched['labels'].shape[1]
    print('Batch: {} \t Batch Size:{} \t Image Size: {} \t Number of Labels: {}'.
          format(i_batch, batch_size, image_size,n_labels))
    

Batch: 0 	 Batch Size:10 	 Image Size: torch.Size([3, 224, 224]) 	 Number of Labels: 9
Batch: 1 	 Batch Size:10 	 Image Size: torch.Size([3, 224, 224]) 	 Number of Labels: 9
Batch: 2 	 Batch Size:10 	 Image Size: torch.Size([3, 224, 224]) 	 Number of Labels: 9
Batch: 3 	 Batch Size:10 	 Image Size: torch.Size([3, 224, 224]) 	 Number of Labels: 9
Batch: 4 	 Batch Size:10 	 Image Size: torch.Size([3, 224, 224]) 	 Number of Labels: 9
Batch: 5 	 Batch Size:10 	 Image Size: torch.Size([3, 224, 224]) 	 Number of Labels: 9
Batch: 6 	 Batch Size:10 	 Image Size: torch.Size([3, 224, 224]) 	 Number of Labels: 9
Batch: 7 	 Batch Size:10 	 Image Size: torch.Size([3, 224, 224]) 	 Number of Labels: 9
Batch: 8 	 Batch Size:10 	 Image Size: torch.Size([3, 224, 224]) 	 Number of Labels: 9
Batch: 9 	 Batch Size:10 	 Image Size: torch.Size([3, 224, 224]) 	 Number of Labels: 9


In [7]:
#No longer needed since we are normalizing based off the pretrained models
#Calculate mean &std
#x=[]
#for i_batch,sample_batched in enumerate(yelpTrainLoader,0):
#    numpy_image = sample_batched['image'].numpy()
#    x.append(np.mean(numpy_image, axis=(0,2,3)))

#image_mean=np.mean(x, axis= (0))
#image_std=np.std(x, axis=(0))

In [8]:
#No longer needed since we are normalizing based off the pretrained models
#Confirm images have been normalized appropriately with 0 mean and 1 std
#print(image_mean)
#print(image_std)

# VGG19 with Batch Normilzation

In [6]:
# Load AlexNet and VGGNet with batch Normilization
import torchvision.models as models
vgg19 = models.vgg19_bn(pretrained=True)

In [7]:
#Modify VGGNET19
#Don't update the existing parameters 

for param in vgg19.parameters():
    param.requires_grad = False

#By default this requires_grad is set to true
vgg19.classifier._modules['6']  = nn.Linear(4096, 9)


In [8]:
# Define hyper-parameters
#******************************#
learning = .01 # Learning Rate - chosen using grid search at end
moment =.9 #Momentum - chosen using grid search at end

#******************************#

In [49]:
# 4.1 Define criterion and optimizer
#************************************#
import torch.optim as optim


criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(vgg19.classifier._modules['6'].parameters(), lr=learning, momentum=moment)

#************************************#

# 4.2 Train the model
# 4.3 Please store and print training and validation loss&accuracy after each epoch
#********************************************#

train_losses = []
test_losses = []
acc_data_train=[]
acc_data_test=[]

def train(epoch):
    vgg19.train()
    train_loss = 0
    TN = 0
    TP = 0
    FP = 0
    FN = 0
    for i, sample_batched in enumerate(yelpTrainLoader,1):
        inputs = sample_batched['image']
        labels = sample_batched['labels']
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg19(inputs)
        loss = criterion(outputs.float(), labels.float())
        loss.backward()
        optimizer.step()
        
        train_loss+= loss.item()
        pred = (torch.sigmoid(outputs).data > 0.5).int()
        #print(pred)
        labels = labels.int()
        #print(labels)
        
        TP += ((pred == 1) & (labels == 1) ).float().sum() #True Positive Count
        TN += ((pred == 0) & (labels == 0) ).float().sum() #True Negative Count
        FP += ((pred == 1) & (labels == 0) ).float().sum() #False Positive Count
        FN += ((pred == 0) & (labels == 1) ).float().sum() #False Negative Count
        #print('TP: {}\t TN: {}\t FP: {}\t FN: {}\n'.format(TP,TN,FP,FN) )    
    
    accuracy = (TP + TN)/(TP + TN + FP + FN)
    precision = TP/(TP + FP)
    recall = TP/(TP + FN)
    f1_score = 2 * (precision*recall)/(precision+recall)
    acc_data_train.append([TP, TN, FP, FN, accuracy, precision, recall, f1_score])
    train_loss=float(train_loss)/float(i)
    train_losses.append(train_loss)
    # print statistics        
    print('Train Epoch:{}  Accuracy: {:.2f}   Average Loss: {:.2f}  Precision: {:.2f}   F1 Score: {:.2f}\n'.
          format(epoch, accuracy, train_loss, precision, f1_score))
    
def test(epoch):
    #Have our model in evaluation mode
    vgg19.eval()
    #Set losses and Correct labels to zero
    test_loss = 0
    TN = 0
    TP = 0
    FP = 0
    FN = 0
    with torch.no_grad():
        for i, sample_batched in enumerate(yelpTestLoader,1):
            inputs = sample_batched['image']
            labels = sample_batched['labels']
            outputs = vgg19(inputs)
            loss = criterion(outputs.float(), labels.float())
            test_loss += loss.item()
            pred = (torch.sigmoid(outputs).data > 0.5).int()
            #print(pred)
            labels = labels.int()
            #print(labels)

            TP += ((pred == 1) & (labels == 1) ).float().sum() #True Positive Count
            TN += ((pred == 0) & (labels == 0) ).float().sum() #True Negative Count
            FP += ((pred == 1) & (labels == 0) ).float().sum() #False Positive Count
            FN += ((pred == 0) & (labels == 1) ).float().sum() #False Negative Count
            #print('TP: {}\t TN: {}\t FP: {}\t FN: {}\n'.format(TP,TN,FP,FN) )    

        accuracy = (TP + TN)/(TP + TN + FP + FN)
        precision = TP/(TP + FP)
        recall = TP/(TP + FN)
        f1_score = 2 * (precision*recall)/(precision+recall)
        acc_data_test.append([TP, TN, FP, FN, accuracy, precision, recall, f1_score])
        test_loss = float(test_loss)/float(i)
        test_losses.append(test_loss)
        # print statistics        
        print('Test Epoch:{}  Accuracy: {:.2f}   Average Loss: {:.2f}  Precision: {:.2f}   F1 Score: {:.2f}\n'.
              format(epoch, accuracy, test_loss, precision, f1_score))

            

In [23]:
#Hyperparameter tuning - Grid Search 

learning_rates = [.1,.01,.001, .00001]
momentums = [.3,.4,.5,.6, .7, .8, .9]
w = [.1,.01,.001]

for i in [1,2]: #Test different optimizer
    for j in range(len(learning_rates)):    
        if i ==1:
            for k in range(len(momentums)):
                #Reset Model per test
                vgg19 = models.vgg19_bn(pretrained=True)
                for param in vgg19.parameters():
                    param.requires_grad = False
                vgg19.classifier._modules['6']  = nn.Linear(4096, 9)
                
                optimizer = optim.SGD(vgg19.classifier._modules['6'].parameters(), 
                                  lr=learning_rates[j], momentum=momentums[k])
                
                print('Optimizer: SGD\tLearning Rate: {}\tMomentum: {}'.
                      format (learning_rates[j], momementums[k]))
                train(epoch)
                
        else: 
            for k in range(len(w)):
                #Reset Model per test
                vgg19 = models.vgg19_bn(pretrained=True)
                for param in vgg19.parameters():
                    param.requires_grad = False
                vgg19.classifier._modules['6']  = nn.Linear(4096, 9)
                
                optimizer = optim.Adam(vgg19.classifier._modules['6'].parameters(), 
                                  lr=learning_rates[j], weight_decay = w[k])
        
                print('Optimizer: Adam\t Learning Rate: {}\t WeightDecay: {}'.
                      format (learning_rates[j], w[k]))
                train(epoch)
            
                

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.3
Train Epoch:1  Accuracy: 0.61   Average Loss: 0.66  Precision: 0.62   F1 Score: 0.63

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.4
Train Epoch:1  Accuracy: 0.60   Average Loss: 0.67  Precision: 0.61   F1 Score: 0.62

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.5
Train Epoch:1  Accuracy: 0.60   Average Loss: 0.70  Precision: 0.61   F1 Score: 0.60

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.6
Train Epoch:1  Accuracy: 0.55   Average Loss: 0.71  Precision: 0.56   F1 Score: 0.58

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.7
Train Epoch:1  Accuracy: 0.64   Average Loss: 0.66  Precision: 0.64   F1 Score: 0.65

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.8
Train Epoch:1  Accuracy: 0.60   Average Loss: 0.75  Precision: 0.60   F1 Score: 0.62

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.9
Train Epoch:1  Accuracy: 0.60   Average Loss: 0.72  Precision: 0.61   F1 Score: 0.61

Optimizer: SGD	Learning Rate: 0.01	Momentum: 0.3
Train Epoch:1

In [50]:
#Finally run model on both training and  test

vgg19 = models.vgg19_bn(pretrained=True)
for param in vgg19.parameters():
    param.requires_grad = False
vgg19.classifier._modules['6']  = nn.Linear(4096, 9)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(vgg19.classifier._modules['6'].parameters(), lr=.1, momentum=.7)

train_losses = []
test_losses = []
acc_data_train=[]
acc_data_test=[]

test(0)
for epoch in range(1,5):
    train(epoch)
    test(epoch)

Test Epoch:0  Accuracy: 0.52   Average Loss: 0.70  Precision: 0.53   F1 Score: 0.56

Train Epoch:1  Accuracy: 0.59   Average Loss: 0.69  Precision: 0.59   F1 Score: 0.62

Test Epoch:1  Accuracy: 0.79   Average Loss: 0.46  Precision: 0.83   F1 Score: 0.78

Train Epoch:2  Accuracy: 0.74   Average Loss: 0.51  Precision: 0.75   F1 Score: 0.75

Test Epoch:2  Accuracy: 0.88   Average Loss: 0.33  Precision: 0.89   F1 Score: 0.88

Train Epoch:3  Accuracy: 0.78   Average Loss: 0.42  Precision: 0.78   F1 Score: 0.79

Test Epoch:3  Accuracy: 0.91   Average Loss: 0.27  Precision: 0.93   F1 Score: 0.91

Train Epoch:4  Accuracy: 0.84   Average Loss: 0.36  Precision: 0.84   F1 Score: 0.85

Test Epoch:4  Accuracy: 0.94   Average Loss: 0.21  Precision: 0.97   F1 Score: 0.94



# Alexnet 

In [66]:
#Alexnet requires 227 x 227
yelpTrainSet = yelpDataset(text_file ='/Users/JosephVele/Downloads/all/train_photo_to_biz_ids.csv',
                           root_dir = '/Users/JosephVele/Downloads/all/train_photos',
                          transform = transforms.Compose([transforms.Resize((227,227)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean = [0.485, 0.456, 0.406],
                                                              std = [0.229, 0.224, 0.225])]))

yelpTrainLoader = torch.utils.data.DataLoader(yelpTrainSet,batch_size=10,shuffle=True, num_workers=0)

yelpTestSet = yelpDataset(text_file ='/Users/JosephVele/Downloads/all/test_photo_to_biz_ids.csv',
                          root_dir = '/Users/JosephVele/Downloads/all/train_photos',
                          transform = transforms.Compose([transforms.Resize((227,227)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean = [0.485, 0.456, 0.406],
                                                              std = [0.229, 0.224, 0.225])]))

yelpTestLoader = torch.utils.data.DataLoader(yelpTrainSet,batch_size=10,shuffle=True, num_workers=0)


In [44]:
alex_net = models.alexnet(pretrained=True)
for param in alex_net.parameters():
    param.requires_grad = False
alex_net.classifier._modules['6'] = nn.Linear(4096, 9)

In [56]:
# 4.1 Define criterion and optimizer
#************************************#
import torch.optim as optim


criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(alex_net.parameters(), lr=learning, momentum=moment)

#************************************#

# 4.2 Train the model
# 4.3 Please store and print training and validation loss&accuracy after each epoch
#********************************************#

train_losses = []
test_losses = []
acc_data_train=[]
acc_data_test=[]

def alex_train(epoch):
    alex_net.train()
    train_loss = 0
    TN = 0
    TP = 0
    FP = 0
    FN = 0
    for i, sample_batched in enumerate(yelpTrainLoader,1):
        inputs = sample_batched['image']
        labels = sample_batched['labels']
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = alex_net(inputs)
        loss = criterion(outputs.float(), labels.float())
        loss.backward()
        optimizer.step()
        
        train_loss+= loss.item()
        pred = (torch.sigmoid(outputs).data > 0.5).int()
        #print(pred)
        labels = labels.int()
        #print(labels)
        
        TP += ((pred == 1) & (labels == 1) ).float().sum() #True Positive Count
        TN += ((pred == 0) & (labels == 0) ).float().sum() #True Negative Count
        FP += ((pred == 1) & (labels == 0) ).float().sum() #False Positive Count
        FN += ((pred == 0) & (labels == 1) ).float().sum() #False Negative Count
        #print('TP: {}\t TN: {}\t FP: {}\t FN: {}\n'.format(TP,TN,FP,FN) )    
    
    accuracy = (TP + TN)/(TP + TN + FP + FN)
    precision = TP/(TP + FP)
    recall = TP/(TP + FN)
    f1_score = 2 * (precision*recall)/(precision+recall)
    acc_data_train.append([TP, TN, FP, FN, accuracy, precision, recall, f1_score])
    train_loss=float(train_loss)/float(i)
    train_losses.append(train_loss)
    # print statistics        
    print('Train Epoch:{}  Accuracy: {:.2f}   Average Loss: {:.2f}  Precision: {:.2f}   F1 Score: {:.2f}\n'.
          format(epoch, accuracy, train_loss, precision, f1_score))
    
def alex_test(epoch):
    #Have our model in evaluation mode
    alex_net.eval()
    #Set losses and Correct labels to zero
    test_loss = 0
    TN = 0
    TP = 0
    FP = 0
    FN = 0
    with torch.no_grad():
        for i, sample_batched in enumerate(yelpTestLoader,1):
            inputs = sample_batched['image']
            labels = sample_batched['labels']
            outputs = alex_net(inputs)
            loss = criterion(outputs.float(), labels.float())
            test_loss += loss.item()
            pred = (torch.sigmoid(outputs).data > 0.5).int()
            #print(pred)
            labels = labels.int()
            #print(labels)

            TP += ((pred == 1) & (labels == 1) ).float().sum() #True Positive Count
            TN += ((pred == 0) & (labels == 0) ).float().sum() #True Negative Count
            FP += ((pred == 1) & (labels == 0) ).float().sum() #False Positive Count
            FN += ((pred == 0) & (labels == 1) ).float().sum() #False Negative Count
            #print('TP: {}\t TN: {}\t FP: {}\t FN: {}\n'.format(TP,TN,FP,FN) )    

        accuracy = (TP + TN)/(TP + TN + FP + FN)
        precision = TP/(TP + FP)
        recall = TP/(TP + FN)
        f1_score = 2 * (precision*recall)/(precision+recall)
        acc_data_test.append([TP, TN, FP, FN, accuracy, precision, recall, f1_score])
        test_loss = float(test_loss)/float(i)
        test_losses.append(test_loss)
        # print statistics        
        print('Test Epoch:{}  Accuracy: {:.2f}   Average Loss: {:.2f}  Precision: {:.2f}   F1 Score: {:.2f}\n'.
              format(epoch, accuracy, test_loss, precision, f1_score))

In [46]:
#Hyperparameter tuning - Grid Search 

learning_rates = [.1,.01,.001, .00001]
momentums = [.3,.4,.5,.6, .7, .8, .9]
w = [.1,.01,.001]

for i in [1,2]: #Test different optimizer
    for j in range(len(learning_rates)):    
        if i ==1:
            for k in range(len(momentums)):
                #Reset Model per test
                alex_net = models.alexnet(pretrained=True)
                for param in alex_net.parameters():
                    param.requires_grad = False
                alex_net.classifier._modules['6'] = nn.Linear(4096, 9)
                
                optimizer = optim.SGD(alex_net.classifier._modules['6'].parameters(), 
                                  lr=learning_rates[j], momentum=momentums[k])
                
                print('Optimizer: SGD\tLearning Rate: {}\tMomentum: {}'.
                      format (learning_rates[j], momementums[k]))
                alex_train(epoch)
                
        else: 
            for k in range(len(w)):
                #Reset Model per test
                alex_net = models.alexnet(pretrained=True)
                for param in alex_net.parameters():
                    param.requires_grad = False
                alex_net.classifier._modules['6'] = nn.Linear(4096, 9)
                
                optimizer = optim.Adam(alex_net.classifier._modules['6'].parameters(), 
                                  lr=learning_rates[j], weight_decay = w[k], amsgrad=True)
        
                print('Optimizer: Adam\t Learning Rate: {}\t WeightDecay: {}'.
                      format (learning_rates[j], w[k]))
                alex_train(epoch)

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.3
Train Epoch:4  Accuracy: 0.54   Average Loss: 1.27  Precision: 0.55   F1 Score: 0.56

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.4
Train Epoch:4  Accuracy: 0.60   Average Loss: 1.09  Precision: 0.61   F1 Score: 0.62

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.5
Train Epoch:4  Accuracy: 0.58   Average Loss: 1.27  Precision: 0.59   F1 Score: 0.59

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.6
Train Epoch:4  Accuracy: 0.58   Average Loss: 1.44  Precision: 0.59   F1 Score: 0.60

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.7
Train Epoch:4  Accuracy: 0.58   Average Loss: 1.41  Precision: 0.59   F1 Score: 0.60

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.8
Train Epoch:4  Accuracy: 0.56   Average Loss: 1.82  Precision: 0.57   F1 Score: 0.57

Optimizer: SGD	Learning Rate: 0.1	Momentum: 0.9
Train Epoch:4  Accuracy: 0.57   Average Loss: 2.10  Precision: 0.58   F1 Score: 0.58

Optimizer: SGD	Learning Rate: 0.01	Momentum: 0.3
Train Epoch:4

In [ ]:
alex_net = models.alexnet(pretrained=True)
for param in alex_net.parameters():
    param.requires_grad = False
alex_net.classifier._modules['6'] = nn.Linear(4096, 9)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(alex_net.classifier._modules['6'].parameters(), 
                                  lr=.001, weight_decay = .1, amsgrad=True)

train_losses = []
test_losses = []
acc_data_train=[]
acc_data_test=[]

alex_test(0)
for epoch in range(1,6):
    alex_train(epoch)
    alex_test(epoch)

Test Epoch:0  Accuracy: 0.51   Average Loss: 0.74  Precision: 0.54   F1 Score: 0.53

Train Epoch:1  Accuracy: 0.69   Average Loss: 0.61  Precision: 0.70   F1 Score: 0.70

Test Epoch:1  Accuracy: 0.72   Average Loss: 0.54  Precision: 0.72   F1 Score: 0.74

Train Epoch:2  Accuracy: 0.69   Average Loss: 0.60  Precision: 0.70   F1 Score: 0.71

Test Epoch:2  Accuracy: 0.72   Average Loss: 0.55  Precision: 0.72   F1 Score: 0.74

Train Epoch:3  Accuracy: 0.69   Average Loss: 0.60  Precision: 0.71   F1 Score: 0.71

